In [1]:
from IPython.core.display import display, HTML, Javascript
from string import Template
import json, os
import pandas as pd
import numpy as np

In [2]:
notebook_path = os.path.abspath("Arrest_Data_toDate.ipynb")
arrest_2020 = os.path.join(os.path.dirname(notebook_path), "data/", "NYPD_Arrest_Data__Year_to_Date_CLEANED.csv")
arrest1 = pd.read_csv(arrest_2020)
arrest_historic = os.path.join(os.path.dirname(notebook_path), "data/", "NYPD_Arrests_Data__2011-2019_CLEANED.csv")
arrest2 = pd.read_csv(arrest_historic)
#arrest1.head()

In [3]:
arrests = arrest2.append(arrest1, ignore_index=True)

In [4]:
def get_month(row):
    arr = row['ARREST_DATE'].split('/')
    return arr[2] + '-' + arr[0]
arrests['MONTH'] = arrests.apply(lambda row: get_month(row), axis=1)

In [5]:
arrest_aggregated = arrests.groupby('MONTH').size().reset_index(name='counts')
#arrest_aggregated.to_csv('arrest_aggregated.csv')
#arrest_aggregated.head()

In [6]:
data = arrest_aggregated.to_dict(orient='records')

In [7]:
%%javascript
require.config({
    paths: {
        d3: "https://d3js.org/d3.v6.min"
     }
});

require(["d3"], function(d3) {
    console.log(d3);
    window.d3 = d3;
});

<IPython.core.display.Javascript object>

In [8]:
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')

In [9]:
css_text = ''''''

In [10]:
js_text_template = Template('''
  var margin = {top: 10, right: 50, bottom: 20, left: 40};
  var visWidth = 740 - margin.left - margin.right;
  var visHeight = 400 - margin.top - margin.bottom;
  
  var svg = d3.select("#$graphdiv")
    .append('svg')
    .attr('width', visWidth + margin.left + margin.right)
    .attr('height', visHeight + margin.top + margin.bottom);   
  
  var g = svg.append('g')
    .attr('transform', `translate(${margin.left}, ${margin.top})`);  
    
  var data1 = $python_data;  
  
  var months = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"];
  

  var list = data1.map(d=>{
    var o = {};
    var time = d.MONTH.split("-");
    o.Month = parseInt(time[1]);
    o.Year = parseInt(time[0]);
    o.count = parseInt(d.counts);
    return o;
  })
  
  var map = {1:"JAN", 2:"FEB", 3:"MAR", 4:"APR", 5:"MAY", 6:"JUN", 7:"JUL", 8:"AUG", 9:"SEP", 10:"OCT", 11:"NOV", 12:"DEC", }
  
  var dict =  d3.rollup(list, v=>v.map(d=>{
    var o = {};
    o.Month = map[d.Month];
    o.Count = d.count;
    return o;
  }), d=>d.Year);
  
  var data = [];
  
  for (var i = 2011; i <= 2020; i++) {
    var o = {};
    o.year = i;
    o.data = dict.get(i);
    data.push(o);
  }
  
  var maxCount = [];
  for (var i = 0; i < data.length; i++) {
    maxCount.push(d3.max(data[i].data, d => d.Count));
  }  
  var MaxCount = d3.max(maxCount);  
  
  var x = d3.scalePoint()
    .domain(months)
    .range([0, visWidth]);
  
  var y = d3.scaleLinear()
    .domain([0, MaxCount]).nice()
    .range([visHeight, 0]);   
  
  var xAxis = d3.axisBottom(x)
  var yAxis = d3.axisLeft(y);   
  
  g.append('g')
    .attr('transform', `translate(0,${visHeight})`)
    .call(xAxis);  
  
  g.append('g')
      .call(yAxis)
    .append('text')
      .attr('text-anchor', 'start')
      .attr('dominant-baseline', 'middle')
      .attr('fill', 'black')
      .attr('x', 5)
      .text('NYC Number of Arrests Made');     
  
  var line = d3.line()
      .x(d => x(d.Month))
      .y(d => y(d.Count));  
  
  var series = g.append('g')
    .selectAll('g')
    .data(data)
    .join('g')
      .attr('stroke', d => d.year == 2020 ? 'red' : 'lightgray')
    .append('path')
      .datum(d => d.data)
      .attr('fill', 'none')
      .attr('stroke-width', 2)
      .attr('d', line);     
  
  g.selectAll('.year-label')
    .data(data)
    .join('text')
      .attr('class', 'year-label')
      .attr('x', d => x(d.data[11].Month) + 2)
      .attr('y', d => y(d.data[11].Count))
      .attr('font-size', 10)
      .attr('font-family', 'sans-serif')
      .attr('dominant-baseline', 'middle')
      .text(d => d.year);    
''')

In [11]:
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')
js_text = js_text_template.safe_substitute(python_data=json.dumps(data),
                                       graphdiv='graph-div')
HTML(html_template.safe_substitute(css_text=css_text, js_text=js_text))